# Обучение модели детектирования Yolov5

Инструкции по установке и подготовке данных находятся в файле ReadMe.md.

In [1]:
# conda activate yolov5

In [ ]:
%cd yolov5

Обучение протестировано на одной видеокарте.

Инструкции по обучению на нескольких видеокартах можно найти в исходном репозитории https://github.com/ultralytics/yolov5/issues/475.

Проверьте, все ли данные подготовлены для обучения, как описано в файле ReadMe.md.

Для обучения необходимы:
- тестовая и валидационная папки с изображениями, на которых необходимо сделать детекции; 
- тестовая и валидационная папки с текстовыми файлами для каждого изображения, в которых указаны лейблы классов и координаты баундинг-боксов;
- конфигурационный файл модели *.yaml; 
- конфигурационный файл данных *.yaml; 
- файл предобученной модели.

Пример команды:

In [ ]:
!python train.py \
    --device 0 \
    --cfg models/yolov5s.yaml \
    --batch-size 16 \
    --img 320 \
    --data data/pipelab_cor.yaml \
    --weights models/yolov5s.pt \
    --name 320 \
    --project runs \
    --epochs 150

Основные аргументы командной строки:

<br>--weights - файл предобученной модели yolov5s.pt, по умолчанию скачивается файл [yolov5s.pt](https://github.com/ultralytics/yolov5/releases/download/v6.2/yolov5s.pt)
<br>--cfg - путь к отредактированному конфигурационному файлу модели
<br>--data - путь к конфигурационному файлу данных
<br>--epochs - количество эпох обучения, по умолчанию - 300
<br>--batch-size - размер батча, по умолчанию - 16
<br>--img-size - размер входного в модель изображения, можно просто --img, по умолчанию: [640, 640] (для квадратных изображений можно указать одним числом)
<br>--device - выбор gpu: 0,1,2,3 or cpu
<br>--single-cls - выбирается, если все объекты считаются одним классом
<br>--project - путь к папке с проектом, куда необходимо сохранить результаты (создается во время обучения), по умолчанию результаты сохраняются в папку runs
<br>--name - имя проекта (создается в папке project во время обучения)
<br>--optimizer - оптимайзер: SGD, Adam или AdamW, по умолчинаю - SGD

В результате обучения будет создана папка, указанная в аргументе --project, в которой находятся сохраненные веса обученной модели, распределение координат рамок по каждому классу, результаты обучения для каждой эпохи, примеры детекций обученной модели.

# Тестирование модели детектирования Yolov5

Для инференса необходимы:
- папка с изображениями, на которых необходимо сделать детекции (точность детектирования будет выше, если масштаб изображений будет совпадать с масштабом изображений, на которых обучалась модель); 
- конфигурационный файл данных *.yaml, с которым обучалась модель; 
- файл обученной модели *.pt.

Инференс можно сделать как на валидационных, так и на тестовых данных. В обоих случаях для этого используется файл val.py. Чтобы выбрать, валидационные или тестовые данные будут использоваться для инференса указывается аргумент командной строки: --task.
<br>Для валидационных данных: --task val
<br>Для тестовых данных: --task test
<br>По умолчанию используется значение val.

Пример команды:

In [ ]:
!python val.py \
    --batch 4 \
    --weights best.pt \
    --data data/data_conf.yaml \
    --img 320 \
    --iou-thres 0.4 \
    --conf-thres 0.4
    --task test \
    --save-conf \
    --save-txt \
    --project runs/test

Основные аргументы командной строки:

<br>--batch - размер партии изображений, которая подается на вход модели
<br>--weights - файл обученной модели
<br>--data - путь к конфигурационному файлу данных
<br>--project - путь к папке, в которую будут сохранены результаты инференса
<br>--img - размер входного в модель изображения, по умолчанию: 640 (для квадратных изображений можно указать одним числом), для прямоугольных - указываются числа через пробел, например: 320 320
<br>--conf-thres - уровень достоверности идентификации классов, по умолчанию 0.001
<br>--iou-thres - уровень достоверности идентификации рамок, по умолчанию 0.6
<br>--device - выбор gpu: 0,1,2,3 или cpu
<br>--save-txt - аргумент, определяющий, будут ли сохраняться текстовые файлы с разметкой детекций для каждого изображения
<br>--task - тип задания: train, val, test, speed или study (мне пригодились только значения val и test)

В результате инференса будет создана папка, указанная в аргументе --project, в которой будут находится изображения с разметкой боксов и текстовые файлы с классами и координатами боксов (как при разметке лейблов для обучения), если указан агрумент --save-txt. 

# Детектирование и вырезание объектов

Для операции вырезания используется скрипт detect.py. Это по сути то же самое, что и инференс, но дополнительно можно сохранить вырезанные детекции в отдельную папку. Изображения вырезает скрипт очень долго. Поэтому проще сделать инференс, и по сохраненным координатам объектов сделать вырезку самостоятельно, например, параллельно.

Пример команды:

In [ ]:
!python detect.py \
    --weights best.pt \
    --source images/test \
    --iou-thres 0.25 \
    --img 320 \
    --save-txt \
    --save-conf \
    --save-crop \
    --project runs/detect

Основные аргументы командной строки:

<br>--weights - файл обученной модели
<br>--source - путь к папке с изображениями, по умолчанию data/images
<br>--project - путь к папке, в которую нужно сохранить детекции, по умолчанию: runs/detect
<br>--img-size - размер входного в модель изображения, можно просто -- img, по умолчанию: 640 640 (для квадратных изображений можно указать одним числом)
<br>--conf-thres - уровень достоверности идентификации классов, по умолчанию 0.25
<br>--iou-thres - уровень достоверности идентификации рамок, по умолчанию 0.45
<br>--device - выбор gpu: 0,1,2,3 или cpu
<br>--save-txt - аргумент, определяющий, будут ли сохраняться текстовые файлы с разметкой детекций для каждого изображения
<br>--save-conf - аргумент, определяющий, будет ли сохраняться уровень достоверности идентификации классов для каждого изображения в текстовый файл (если указано --save-txt)
<br>--save-crop - аргумент, определяющий, будут ли сохраняться вырезанные изображения

В результате детектирования будет создана папка, указанная в аргументе --project, в которой будут находится папки с лейблами и вырезанными изображениями, если указан агрумент --save-crop. 

С дополнительными аргументами командной строки можно ознакомиться в файлах train.py, val.py, detect.py.